In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os
from tensorflow.keras.models import load_model

# Setting path for dataset
train_data_dir = 'D:/python/archive (1)/test'
validation_data_dir = 'D:/python/archive (1)/validation'

# Check the data directory structure
print("Train Directory Contents:", os.listdir(train_data_dir))
print("Validation Directory Contents:", os.listdir(validation_data_dir))

# Create ImageDataGenerator objects for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.3,
    zoom_range=0.4,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Load training data from the directory with chunking
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150, 150),
    batch_size=32,  # Adjust batch size for chunking
    class_mode='categorical'
)

# Load validation data with chunking
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(150, 150),
    batch_size=32,  # Adjust batch size for chunking
    class_mode='categorical'
)

# Initialize the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Check the steps per epoch
print(f"Steps per epoch: {len(train_generator)}")
print(f"Validation steps: {len(validation_generator)}")

# Train the model with chunking
print("Starting model training...")
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=3,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# Evaluate the model on validation data
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {validation_loss}")
print(f"Validation Accuracy: {validation_accuracy}")

# Save the trained model
model.save('brain_tumor_cnn_model.h5')

model = load_model('brain_tumor_cnn_model.h5')
model.summary()

# Function to plot images
def plot_images(generator):
    # Get a batch of images and labels
    images, labels = next(generator)
    
    # Create a figure
    plt.figure(figsize=(10, 10))
    
    # Plot 9 images
    for i in range(9):
        plt.subplot(3, 3, i+1)
        plt.imshow(images[i])
        plt.title(f"Label: {np.argmax(labels[i])}")
        plt.axis('off')
    
    plt.show()

# Plot images from the training dataset
plot_images(train_generator)

# Plot images from the validation dataset
plot_images(validation_generator)

# Visualize training performance
def plot_performance(history):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    plt.show()

# Call the performance plot function
plot_performance(history)
